In [23]:
import os
os.chdir('/Users/likechen/Desktop/GitHub/George/George-portfolio_optimization/data')
import numpy as np
import pandas as pd
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import expected_returns, risk_models

In [58]:
class PortfolioOptimizer:
    def __init__(self, tickers:list, start_date:str, end_date:str):
        """
        initialize portfolio optimizer
        :tickers : (e.g. ['0001.HK','0002.HK'])
        :start_date: (e.g. '2020-01-01')
        :end_date: (e.g. '2025-01-01')
        """
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.prices = self.fetch_prices()
        self.returns = self.calculate_logreturns()
        self.mu = expected_returns.mean_historical_return(self.prices, log_returns=True)
        self.S = risk_models.sample_cov(self.prices, log_returns=True)
        self.weights = None
        
    def fetch_prices(self):
        """
        fetch prices from data file and concatenate them into one dataframe
        (Adj Close = Close * Adjustment Factor)
        """
        adj_close_dfs = []
        for ticker in self.tickers:
            df = pd.read_csv(f'{ticker}.csv', index_col='Date', parse_dates=True)
            df.index = df.index.tz_localize(None)
            start_date = pd.to_datetime(self.start_date).tz_localize(None)
            end_date = pd.to_datetime(self.end_date).tz_localize(None)
            mask = (df.index >= start_date) & (df.index <= end_date)
            df = df.loc[mask]
            df['Adjustment Factor'] = (1 - df['Dividends'] / df['Close']).cumprod()
            df['Adj Close'] = df['Close'] * df['Adjustment Factor']
            adj_close_dfs.append(df[['Adj Close']].rename(columns={'Adj Close': ticker}))
        return pd.concat(adj_close_dfs, axis=1).dropna(how='all')
 
    def calculate_logreturns(self):
        """return -> log returns"""
        return np.log(self.prices / self.prices.shift(1)).dropna()
    
    def optimize(self, objective, risk_free_rate=0.0, target_return=None):
        """
        run the optimizer
        :objective ('min_variance', 'max_sharpe', 'Markowitz')
        :risk_free_rate (annualized)
        :target_return (daily for Markowitz）
        return -> {ticker: weight}
        """
        ef = EfficientFrontier(self.mu, self.S, weight_bounds=(0, 1))

        if objective == 'min_variance':
            weights = ef.min_volatility()
        elif objective == 'max_sharpe':
            weights = ef.max_sharpe(risk_free_rate=risk_free_rate)
        elif objective == 'Markowitz':
            if target_return is None:
                raise ValueError("Markowitz optimization requires target_return")
            weights = ef.efficient_return(target_return=target_return)
        else:
            raise ValueError(f"Unknown objective: {objective}")

        self.weights = ef.clean_weights()
        return self.weights

    def portfolio_performance(self, risk_free_rate=0.0):
        """
        output optimized performance
        :risk_free_rate (annualized)
        """
        if self.weights is None:
            raise ValueError("No weights available. Run optimize() first.")
    
        ef = EfficientFrontier(self.mu, self.S, weight_bounds=(0, 1))
        weights_array = np.array([self.weights[ticker] for ticker in self.tickers])
        ef.weights = weights_array
        
        return ef.portfolio_performance(risk_free_rate=risk_free_rate, verbose=True)

In [59]:
portfolio_1 = PortfolioOptimizer(['0001.HK', '0002.HK', '0003.HK'],'2024-01-01','2025-05-01')
all_adj_close = portfolio_1.fetch_prices()
logreturn=portfolio_1.calculate_logreturns()
print(all_adj_close.head())
print(logreturn.head())

              0001.HK    0002.HK   0003.HK
Date                                      
2024-01-02  38.834469  59.775291  5.600152
2024-01-03  38.928612  59.588203  5.515159
2024-01-04  38.599110  59.822063  5.534047
2024-01-05  38.410820  59.728519  5.571822
2024-01-08  38.269604  59.073708  5.534047
             0001.HK   0002.HK   0003.HK
Date                                    
2024-01-03  0.002421 -0.003135 -0.015293
2024-01-04 -0.008500  0.003917  0.003419
2024-01-05 -0.004890 -0.001565  0.006803
2024-01-08 -0.003683 -0.011024 -0.006803
2024-01-09  0.009792  0.003951  0.010187


In [69]:
min_var_weights = portfolio_1.optimize('min_variance')
max_sharpe_weights = portfolio_1.optimize('max_sharpe', risk_free_rate=0.03)  
Markowitz_weights = portfolio_1.optimize('Markowitz', target_return=0.05)
print(min_var_weights)
print(max_sharpe_weights)
print(Markowitz_weights)

OrderedDict({'0001.HK': 0.02582, '0002.HK': 0.6633, '0003.HK': 0.31088})
OrderedDict({'0001.HK': 0.0, '0002.HK': 0.0, '0003.HK': 1.0})
OrderedDict({'0001.HK': 0.00512, '0002.HK': 0.59766, '0003.HK': 0.39722})


In [70]:
performance = portfolio_1.portfolio_performance(risk_free_rate=0.02)

Expected annual return: 5.0%
Annual volatility: 16.8%
Sharpe Ratio: 0.18
